In [15]:
# cd.. In separate cell if used

from lib.network_mapper import network_mapper_function
from lib.NR3 import NR3_function
#from lib.computeFTreal import computeFTrealfunction
#from lib.computeJTreal import computeJTrealfunction
#from lib import *
import numpy as np
import math
import matplotlib.pyplot as plt
#matplotlib inline
fig_size = np.array([18, 6])
plt.rcParams["figure.figsize"] = fig_size

# z = exp(s*T)

# zoh
# z ~ 1 + s*T
# s ~ (z - 1)/T

# bilinear transform
# z ~ (2 + sT)/(2 - sT)
# s ~ (2/T)(z - 1)/(z + 1)

In [16]:
fp = '/home/michael/Desktop/git/LinDist3Flow/20180601/NETWORKS/'
fn = '03node_fullphase_radial_example.txt'
#fn = '05node_fullphase_radial.txt'

#fn = '05node_fullphase_radial.txt'

network1 = network_mapper_function(fp, fn)

nnode1 = network1.nodes.nnode
nline1 = network1.lines.nline

# Network paramaters

nnode = network1.nodes.nnode
nline = network1.lines.nline

# Load parameters

network1.loads.aPQ = 0.75*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.1*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.15*np.ones((3,nnode1))*(network1.loads.spu != 0)

network1.loads.aPQ = 1.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)

#network1.loads.spu = 1.25*network1.loads.spu

# Capacitor parameters

network1.caps.cappu = 1*network1.caps.cappu

# Controller parameters

network1.cons.wmaxpu = 0.5*network1.cons.wmaxpu


BASE
Vbase: 1.0
Sbase: 10.0
Ibase: 10.0
Zbase: 0.1

NODES
nnode: 4
nodelist: ['A0', 'A1', 'A2', 'A3']
phases: ['abc', 'abc', 'abc', 'abc']
PH:
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]

LINES
nline: 3
TXnode: ['A0', 'A1', 'A1']
RXnode: ['A1', 'A2', 'A3']
TXnum: [0 1 1]
RXnum: [1 2 3]
phases: ['abc', 'abc', 'abc']
PH:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
config: ['12', '12', '12']
length: [0.0005 0.001  0.001 ]

NODES + LINES
inlines:
 [[-1  0  1  2]]
innodes:
 [[-1  0  1  1]]
outlines:
 [[ 0  1 -1 -1]
 [-1  2 -1 -1]]
outnodes:
 [[ 1  2 -1 -1]
 [-1  3 -1 -1]]

CONFIGS
nconf: 1
conflist: ['12']
12 - FZpl:
 [[10. +25.j  1.5 +5.j  1.5 +5.j]
 [ 1.5 +5.j 10. +30.j  1.5 +5.j]
 [ 1.5 +5.j  1.5 +5.j 10. +30.j]]

IMPEDANCE
0 - FZpu:
 [[0.005  +0.0125j 0.00075+0.0025j 0.00075+0.0025j]
 [0.00075+0.0025j 0.005  +0.015j  0.00075+0.0025j]
 [0.00075+0.0025j 0.00075+0.0025j 0.005  +0.015j ]]
1 - FZpu:
 [[0.01  +0.025j 0.0015+0.005j 0.0015+0.005j]
 [0.0015+0.005j 0.01  +0.03j  0.0015+0.005j]
 [0.0015+0.005j 0.0015+0.0

In [17]:
network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(network1,slackidx,Vslack,None,None,None,None)

'''
VNR = np.round(VNR,decimals=6)
INR = np.round(INR,decimals=6)
STXNR = np.round(STXNR,decimals=6)
SRXNR = np.round(SRXNR,decimals=6)
iNR = np.round(iNR,decimals=6)
sNR = np.round(sNR,decimals=6)
'''

print('VNR\n', np.round(VNR,decimals=6))
print('INR\n', np.round(INR,decimals=6))
print('STXNR\n', np.round(STXNR,decimals=6))
print('SRXNR\n', np.round(SRXNR,decimals=6))
print('iNR\n', np.round(iNR,decimals=6))
print('sNR\n', np.round(sNR,decimals=6))
print('itercount:', itercount)

print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

VNR
 [[ 1.      +0.j        0.990827-0.009719j  0.98476 -0.016211j
   0.978547-0.022666j]
 [-0.5     -0.866025j -0.507225-0.84394j  -0.512131-0.829367j
  -0.516768-0.814341j]
 [-0.5     +0.866025j -0.488124+0.865205j -0.480233+0.864682j
  -0.472264+0.864085j]]
INR
 [[ 1.216778-0.331407j  0.404409-0.108205j  0.812369-0.223202j]
 [-1.34981 -0.981877j -0.444086-0.328647j -0.905724-0.653229j]
 [-0.172878+0.946318j -0.058879+0.314247j -0.113998+0.63207j ]]
STXNR
 [[1.216778+0.331407j 0.401751+0.103282j 0.807086+0.213259j
  0.      +0.j      ]
 [1.525235+0.678031j 0.50261 +0.208084j 1.010692+0.433042j
  0.      +0.j      ]
 [0.905974+0.323442j 0.300629+0.102449j 0.602516+0.209897j
  0.      +0.j      ]]
SRXNR
 [[1.208837+0.31654j  0.4     +0.1j      0.8     +0.2j
  0.      +0.j      ]
 [1.513302+0.641126j 0.5     +0.2j      1.      +0.4j
  0.      +0.j      ]
 [0.903144+0.312346j 0.3     +0.1j      0.6     +0.2j
  0.      +0.j      ]]
iNR
 [[ 0.      -0.j        0.      -0.j        0.404409-

In [18]:
fid = open('NR3-' + fn[:-4] + '-python.txt','w')

phstr = ['a','b','c']

fid.write('VNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(VNR[ph,k1].real) + ' ' + '{0:.6f}'.format(VNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nINR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(INR[ph,k1].real) + ' ' + '{0:.6f}'.format(INR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nSTXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(STXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(STXNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nSRXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(SRXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(SRXNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\niNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(iNR[ph,k1].real) + ' ' + '{0:.6f}'.format(iNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nsNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(sNR[ph,k1].real) + ' ' + '{0:.6f}'.format(sNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.close()